In [9]:
import numpy as np
import pandas as pd

Import the data sets: prosecutor, sentencing, flagged cases and policy-eligible statutes.
Define all the policy-eligible statutes based on the document provided by the District Attorney's Office. Each list contains statutes that apply to a different section fo the policy.

In [10]:
pros = pd.read_csv('/Users/clairefenton/Downloads/2023-03-02_name_removed/Emory-Prosecutor Data-02-23-2023_name_rem.csv')
sent = pd.read_csv('/Users/clairefenton/Downloads/2023-03-02_name_removed/Emory-Sentencing Information-02-27-2023_name_rem.csv')
flagged = pd.read_csv('/Users/clairefenton/Downloads/2023-03-02_name_removed/Flagged Cases-02-23-2023_name_rem.csv')
vgsca = ['16-13-30a', '16-13-30b', '16-13-30l2', '16-13-30i1', '16-13-30i2', '16-13-24', '16-13-26', '16-13-1c', '16-13-1b', '16-13-3', '16-13-21', '16-13-30.1', '16-13-30.2']
forgery = ['16-9-1a', '16-9-1b', '16-9-1c', '16-9-1d', '16-9-1e']
fraud = ['16-9-121a1', '16-9-121a2', '16-9-121a3', '16-9-121a4', '16-9-121a5', '16-9-33', '16-9-31']
theft = ['16-18-4']
burglary = ['16-7-1b', '16-7-1c', '16-8-18']

Combine all the eligible statutes across the different policy sections.

In [11]:
eligible = vgsca + forgery + fraud + theft + burglary

Merging the prosecutor and sentencing data sets using a left join.

In [12]:
sent['Matching_ID'] = sent.CaseNbr + sent.PartyID.astype(str)
pros['Matching_ID'] = pros.CaseNumber + pros.UniquePersonID.astype(str)

Merged = sent.merge(pros, on='Matching_ID', how='left')
Merged.to_csv("test.csv")

col_values = []
for col in Merged:
    col_values.append(Merged[col].unique())

col_names = []
for col in Merged.columns:
    col_names.append(col)

uniqe_value = {col_names[i]: col_values[i] for i in range(len(col_names))}

In [13]:
Merged.Degree.value_counts()

Misdemeanor       693922
Felony            389119
X                  39668
Serious Felony     29652
Ordinance            396
Name: Degree, dtype: int64

There are 389,119 felony charges in the joined data set.

Define a function for cleaning the statutes to account for human error. Removes all whitespaces, parentheses, extraneous dashes and sets all the letters to lower case.

In [14]:
def formatStatute(statute):
    statute = statute.lower()
    statute = statute.replace('(', '').replace(')', '').replace('s', '').replace(' ', '')
    if statute.count('-') >= 3:
        index = -1
        while statute.count('-') > 2:
            if statute[index] == '-':
                statute = statute[:index] + statute[(index + 1):]
            else:
                index = index - 1
    return statute 

Clean all the statutes in the merged data set.

In [15]:
messyStatutes = Merged.Statute.copy()
for i in range(0, len(messyStatutes)):
    messy = messyStatutes[i]
    if type(messyStatutes[i]) != float:
        messyStatutes[i] = formatStatute(messy)

Add the cleaned statutes back into the merged data set as a separate column, cleanedStatute.

In [16]:
Merged['cleanedStatute'] = messyStatutes

In [17]:
print(len(flagged[flagged['CaseFlagWord'] == 'CBL']['ControlNum'].unique()))
print(len(flagged[flagged['CaseFlagWord'] == 'CBL']))

1993
2119


There are 1993 unique case numbers in the flagged data set and 2119 observations flagged with the CBL tag (policy eligible).

After merging the flagged data set, we end up with a total of 1,154,945 observations.

In [18]:
result = pd.merge(flagged, Merged, left_on = 'ControlNum', right_on = 'CaseNumber', how = 'right')
print(len(result))

1154945


In [19]:
markedCBL = result[result['CaseFlagWord'] == 'CBL'].cleanedStatute.unique()
print(len(markedCBL))
print(len(eligible))

233
29


There are 233 different statutes represented in the cases flagged as CBL, which is far more than the 29 included in the official statute list from the DA's Office.

In [20]:
cbl = flagged[flagged['CaseFlagWord'] == 'CBL']

In [21]:
unaccountedFor = []
for case in list(cbl['ControlNum']):
    if case not in list(Merged['CaseNumber']):
        unaccountedFor.append(case)
len(unaccountedFor)

357

There are 357 case numbers marked as CBL that do not appear in the prosecuting and sentencing data set.

Specify the charge dispositions which indicate the policy was applied. Examine all the different dispositions included in the entire data set and those applied to the CBL cases to ensure all the correct ones are accounted for. 

In [22]:
result[result['CaseFlagWord'] == 'CBL']['ChargeDisposition'].unique()

array(['Nolle Prosse - Restrict',
       'Not Presented to Grand Jury - Restrict', nan,
       'Not Presented to Grand Jury', 'Pre Trial Diversion - Restrict',
       'Not Guilty - Restrict', 'Dismissed', 'Nolle Prosse', 'Guilty',
       'Dead Docket', 'First Offender Act - Pending',
       'First Offender Act Completed',
       'No Further Action Anticipated - Restrict',
       'Not Guilty by Reason of Insanity - Restrict',
       'Conditional Discharge Pending',
       'Conditional Discharge - Completed'], dtype=object)

In [23]:
result['ChargeDisposition'].unique()

array(['Not Presented to Grand Jury', 'First Offender Act - Pending',
       'Not Guilty - Restrict', 'Nolle Prosse', 'Guilty', nan,
       'Nolle Prosse - Restrict', 'Merged',
       'First Offender Act Completed', 'Nolo',
       'Immunity From Prosecution',
       'Not Presented to Grand Jury - Restrict',
       'Conditional Discharge Pending', 'Jury Verdict Not Guilty',
       'Dead Docket', 'Jury Verdict Guilty (Convicted/Adjudicated)',
       'Dismissed', 'Directed Verdict', 'Not Guilty',
       'Conditional Discharge - Completed',
       'Pre Trial Diversion - Restrict', 'Amended',
       'First Offender Act Revoked', 'No Bill - Restrict',
       'First Offender Act Tolled', 'No Accusation Filed',
       'Guilty by Mentally Insane', 'Vacated', 'Discharged',
       'Transferred to Solicitor General Office',
       'Dismissed - Completed Pretrial Diversion',
       'Jury Verdict Not Guitly by Reason of Insanity',
       'Ameded - Restrict', 'Transfer to Other Jurisdiction',
       

In [24]:
statutes = eligible
#dispositions = ['Nolle Prosse', 'Nolle Prosse - Converted', 'Nolle Prosse - Restrict', 'Not Presented to Granted Jury - Restrict - Converted', 'Not Presented to Grand Jury', 'Not Presented to Grand Jury - Restrict']
dispositions = ['Nolle Prosse', 'Dismissed', 'Dismissed - Restrict', 'Dismissed - Restricted', 'Dismissed - Converted', 'Nolle Prosse', 'Nolle Prosse - Converted', 'Nolle Prosse - Restrict', 'Not Presented to Granted Jury - Restrict - Converted', 'Not Presented to Grand Jury', 'Not Presented to Grand Jury - Restrict', 'Pre Trial Diversion - Restrict']

Serious felonies as specified in the list which the DA's Office provided. 

In [25]:
svf = ['16-10-32a3', '16-10-70b', '16-11-1', '16-11-133b', '16-11-160', '16-11-1602bi', '16-11-160a2', '16-11-160a1', '16-11-160c', '16-11-1a', '16-13-30b', '16-13-2012', '16-13-31b', '16-13-31d', '16-13-31f', '16-13-31.1a', '16-5-1a', '16-5-1c', '16-5-1d', '16-5-24a', '16-5-24c1', '16-5-24g', '16-5-40a', '16-5-44', '16-5-44.1(b)(1)', '16-5-44.1(b)(2)', '16-5-46(c)', '16-5-72(a)', '16-5-80(b)', '16-6-1(a)', '16-6-2(a)(2)', '16-6-22', '16-6-22.2(b)', '16-6-22a', '16-6-3(a)', '16-6-4(c)', '16-6-5(a)', '16-7-5(b)', '16-7-88(a)', '16-7-88(b)', '16-8-41(a)', '16-8-41(c)(1)', '17-10-16a', '17-10-6.1b1-2', '17-10-7', '17-10-7c']
for i in range(0, len(svf)):
    messy = svf[i]
    svf[i] = formatStatute(messy)

Create a dictionary with each Matching_ID keys and add three items as values:
1. A list of all the statutes corresponding to the Matching_ID (i.e. person + case combination)
2. A count of all the corresponding statutes which are policy eligible
3. A count of all the corresponding statutes which are policy eligible and fall under Item 4 (fraud)

In [26]:
defendants = {}
subSent = result[['Matching_ID', 'cleanedStatute']]
for i in range(0, len(subSent)):
    if subSent.loc[i][0] not in defendants:
        defendants[subSent.loc[i][0]] = [[subSent.loc[i][1]], 0, 0]
    else:
        defendants[subSent.loc[i][0]][0].append(subSent.loc[i][1])
    if subSent.loc[i][1] in eligible:
        defendants[subSent.loc[i][0]][1] = defendants[subSent.loc[i][0]][1] + 1
    if subSent.loc[i][1] in fraud:
        defendants[subSent.loc[i][0]][2] = defendants[subSent.loc[i][0]][2] + 1

Determine eligibility based on following critieria:
1. Misdemeanor
2. Item 4 charges where only other felony charges are other Item 4 charges
3. Felony charges in specified statutes where the given charge is the only one

In [27]:
policy_case = result[['cleanedStatute', 'ChargeDisposition', 'Matching_ID', 'Degree']]
flag = []
Treatment = []
iteration = list(range(0, len(policy_case)))

for i in iteration:
    if (policy_case['Degree'][i] == 'Misdemeanor') | ((policy_case['cleanedStatute'][i] in fraud) & (defendants[policy_case['Matching_ID'][i]][2] == len(defendants[policy_case['Matching_ID'][i]][0]))) | ((policy_case['cleanedStatute'][i] in eligible) & (len(defendants[policy_case['Matching_ID'][i]][0]) == 1) & (defendants[policy_case['Matching_ID'][i]][1] == 1)):
        flag.append(True)
        if policy_case['ChargeDisposition'][i] in dispositions:
            Treatment.append(True)
        else:
            Treatment.append(False)
    else:
        flag.append(False)
        Treatment.append(False)

In [28]:
result['CaseFlag'] = flag
result['PolicyApplied'] = Treatment

In [29]:
print("Eligible Charges", sum(flag))
print("Applied Charges", sum(Treatment))

Eligible Charges 700789
Applied Charges 217375


Including misdemeanors, which make up the majority of charges, there are 700,789 charges eligible for dismissal under the policy and 217,375 had the policy applied. Since misdemeanors were already up for dismissal before the policy implementation, we want to examine the non-violent felonies only to see how the policy is put into practice with the statutes specified by the DA's Office.

In [30]:
onlyFel = result[(result['Degree'] != 'Misdemeanor') & (result['CaseFlag'] == True)]
print("Eligible Charges", sum(onlyFel['CaseFlag']))
print("Applied Charges", sum(onlyFel['PolicyApplied']))

Eligible Charges 6634
Applied Charges 2246


We estimated that 6634 cases are eligible for the policy, and there are 4259 cases that are actually flagged in the data set as CBL. Out of those 6634 eligible cases, only 2246 had the policy applied. Again, there appears to be something wrong with our disposition criteria — additionally, part of the issue may arise from the fact that the overlap between CBL flagged cases and the entire data set is slim.

In [ ]:
onlyFel[onlyFel['CaseFlag'] == True]

In [33]:
import datetime
import math

Format the DateFiled, CaseOriginDate and DispositionDate columns as datetime objects and re-append them to the data frame as FormattedDateFiled, FormattedCaseOriginDate and FormattedDispositionDate.

In [35]:
FormattedDispositionDate = pd.to_datetime(result['DispositionDate'])
FormattedCaseOriginDate = pd.to_datetime(result['CaseOriginDate'])
FormattedDateFiled = pd.to_datetime(result['DateFiled'])

In [36]:
result['FormattedDispositionDate'] = FormattedDispositionDate
result['FormattedCaseOriginDate'] = FormattedCaseOriginDate
result['FormattedDateFiled'] = FormattedDateFiled

In [ ]:
result[['DispositionDate', 'FormattedDispositionDate', 'CaseOriginDate', 'FormattedCaseOriginDate', 'DateFiled', 'FormattedDateFiled']]

Sort the data set by FormattedDate (a standardized CaseOriginDate) from earliest to latest.

In [38]:
result = result.sort_values(by='FormattedCaseOriginDate')

In [ ]:
result.FormattedCaseOriginDate

In [44]:
cleaned = pd.read_csv('/Users/clairefenton/Desktop/Emory/cleanedData.csv')
earlierArrest = cleaned['EarlierArrest']
result['EarlierArrest'] = earlierArrest
result.to_csv('cleanedData.csv')

/Users/clairefenton/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,5,6,17,18,24,26,27,28,29,30,31,33,34,35,36,37,38,39,40,44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Create recidivism variable that identifies defendants who have previously appeared in the data set according to the CaseOriginDate

Loop through every observation, append the PartyID to the recidivismDefendants list if not already added and append a False value to earlierArrest if so. If the defendant already appeared in the data set, append a True value to earlierArrest. Append an nan value to earlierArrest if the data is also an nan value.

In [ ]:
subSent = result[['PartyID', 'Matching_ID', 'CaseOriginDate']]
allParties = subSent.PartyID.unique()
allParties = list(allParties)

In [ ]:
visitedCases = []
earlierArrest = []

for i in range(0, len(subSent)):
    if subSent['PartyID'][i] in allParties:
        visitedCases.append(subSent['Matching_ID'][i])
        delete = allParties.index(subSent['PartyID'][i])
        del allParties[delete]
        earlierArrest.append(False)
    elif subSent['Matching_ID'][i] in visitedCases:
        earlierArrest.append(False)
    else:
        earlierArrest.append(True)

old code is below, not used but kept just in case

In [ ]:
result['EarlierArrest'] = earlierArrest

In [ ]:
check = result[['CaseNbr', 'PartyID', 'ChargeID', 'Degree', 'cleanedStatute', 'CaseFlagWord', 'CaseFlag', 'CaseOriginDate']]
check[(check['CaseFlag'] == True) & (check['Degree'] != 'Misdemeanor') & (check['CaseFlagWord'] == 'CBL')]

,CaseNbr,PartyID,ChargeID,Degree,cleanedStatute,CaseFlagWord,CaseFlag,CaseOriginDate
892851,12CR6209,1718296,854682,Felony,16-9-31,CBL,True,6/22/2012
892849,12CR6209,1718296,854680,Felony,16-9-31,CBL,True,6/22/2012
892850,12CR6209,1718296,854681,Felony,16-9-31,CBL,True,6/22/2012
898367,12CR6209,1718296,854679,Felony,16-9-31,CBL,True,6/22/2012
965033,16CR2432,8016752,1648804,Felony,16-13-30a,CBL,True,3/9/2016
...,...,...,...,...,...,...,...,...
80643,D0291169,1649383,4782221,Felony,16-13-30a,CBL,True,8/8/2022
85215,22CR2730,10277182,4790462,Felony,16-13-30b,CBL,True,9/18/2022
81509,22CR2543,10272581,4778536,Felony,16-13-30a,CBL,True,9/19/2022
92711,D0292642,10359033,4800248,Felony,16-13-30a,CBL,True,11/8/2022


There are 321 cases where we are perfectly aligned: we marked as eligible and they are CBL in the CaseFlagWord column.

In [ ]:
allEligibleFel = check[((check['CaseFlag'] == True) | (check['CaseFlagWord'] == 'CBL')) & (check['Degree'] != 'Misdemeanor')]
allEligibleFel

,CaseNbr,PartyID,ChargeID,Degree,cleanedStatute,CaseFlagWord,CaseFlag,CaseOriginDate
1003827,81CR2622,6588249,4699770,Felony,16-9-31,NaN,True,2/6/1981
1003905,82CR1481,6596034,3291526,Felony,16-9-31,NaN,True,12/16/1981
1003909,82CR1703,6599125,3349483,Felony,16-9-31,NaN,True,12/28/1981
1003968,83CR1117,6603276,3417258,Serious Felony,16-13-30b,NaN,True,9/13/1982
1004044,83CR2304,6613108,3162609,Felony,16-9-33,NaN,True,4/26/1983
...,...,...,...,...,...,...,...,...
1137480,D00231098,8099304,1529979,Felony,16-9-33,NaN,True,NaN
1140413,D00252888,5954798,1532915,Felony,16-9-33,NaN,True,NaN
1142388,D0034157,6076097,1534888,Felony,16-9-33,NaN,True,NaN
1146102,D0067400,6024108,1538602,Felony,16-8-18,NaN,True,NaN


There are 9136 non misdemeanor cases which are either marked as CBL or we marked as eligible.

In [ ]:
flaggedTrue = allEligibleFel[allEligibleFel['CaseFlag'] == True]
flaggedTrue.CaseFlagWord.value_counts()

CBL        321
DCAP       119
DCAPREF     17
Name: CaseFlagWord, dtype: int64

There are only 321 total CBL flags appearing in all the 9136 non misdemeanor charges we flagged as eligible, and only 457 total CaseFlagWords. Most of the cases we flagged as eligible do not have any CaseFlagWord data for us to compare against.

In [ ]:
cblEligible = result[['CaseNbr', 'PartyID', 'ChargeID', 'Degree', 'cleanedStatute', 'CaseFlagWord', 'CaseFlag', 'PolicyApplied', 'CaseOriginDate']]
cblEligible = cblEligible[cblEligible['CaseFlagWord'] == 'CBL']
cblEligible

,CaseNbr,PartyID,ChargeID,Degree,cleanedStatute,CaseFlagWord,CaseFlag,PolicyApplied,CaseOriginDate
1061803,95CR2704,8025493,4587321,Felony,17-6-12,CBL,False,False,5/8/1995
1061802,95CR2704,8025493,1331225,Felony,2404,CBL,False,False,5/8/1995
859906,03CR1026,2647627,4139159,Felony,42-8-38,CBL,False,False,1/6/2003
315693,03CR1026,2647627,229705,Felony,16-13-30,CBL,False,False,1/6/2003
850360,06CR2776,204255,4680922,Felony,17-6-12,CBL,False,False,5/26/2006
...,...,...,...,...,...,...,...,...,...
83483,D0292874,8201802,4795590,Misdemeanor,16-11-39,CBL,True,True,NaN
89657,D0288914,9110955,4756853,Misdemeanor,40-5-20a,CBL,True,True,NaN
530926,06CR4865,8353585,448814,Felony,16-13-30,CBL,False,False,NaN
849119,06CR4865,8353585,4586648,Felony,17-6-12,CBL,False,False,NaN


In [ ]:
cblEligible.cleanedStatute.unique()

There are many statutes corresponding with CBL cases that are not included in the list of statutes provided by the DA's Office. Such instances might be examples of prosecutors using discretion and dismissing non-eligible cases "in the spirit of the policy." These statutes could also correspond to midemeanor offenses, which would not appear on the list provided by the DA's Office but would still be policy eligible.

The cblEligible subset is what must be used to calculate the applied rate since some cases do not have a CaseFlagWord variable.

In [ ]:
print("Eligible Charges", sum(cblEligible.CaseFlag))
print("Applied Charges", sum(cblEligible.PolicyApplied))
print("Percentage of CBL charges deemed eligible", sum(cblEligible.CaseFlag)/len(cblEligible))
print("Percentage of CBL charges deemed applied", sum(cblEligible.PolicyApplied)/len(cblEligible))

Eligible Charges 1757
Applied Charges 1576
Percentage of CBL charges deemed eligible 0.41253815449636067
Percentage of CBL charges deemed applied 0.37003991547311577
Percentage of CBL cases which are not midemeanors 0.6628316506222118


In [ ]:
print("Percentage of CBL cases which are not midemeanors", len(cblEligible[cblEligible['Degree'] != 'Misdemeanor'])/len(cblEligible))
print("Percentage of eligible CBL cases which are not misdemeanors", len(cblEligible[(cblEligible['Degree'] != 'Misdemeanor') & (cblEligible['CaseFlag'] == True)])/len(cblEligible))
print("Percentage of applied CBL cases which are not misdemeanors", len(cblEligible[(cblEligible['Degree'] != 'Misdemeanor') & (cblEligible['CaseFlag'] == True) & (cblEligible['PolicyApplied'] == True)])/len(cblEligible))

Percentage of CBL cases which are not midemeanors 0.6628316506222118
Percentage of eligible CBL cases which are not misdemeanors 0.07536980511857244
Percentage of applied CBL cases which are not misdemeanors 0.06550833528997417


In [ ]:
result.to_csv('cleanedData.csv')